In [2]:
import pandas as pd
import json
import plotly.express as px
import numpy as np
import scipy.stats as st
from util import open_largest_file_in_zip

In [3]:
path = '../private_data/gr14_report/csv_output_20220629-224721_csv_output.csv'
output_df = pd.read_csv(path).set_index('handle')
output_df.head(2)

,aggregate_score,prediction_score,evaluation_score,heuristic_score,create_distance,update_distance,public_repos,followers,following,bio_length,amount_in_usdt_median,amount_in_usdt_std,amount_in_usdt_std_per_count,amount_to_gitcoin_ratio,contrib_count,eth_share,token
handle,,,,,,,,,,,,,,,,,
ahazel51874,0.0,0.182424,NaN,NaN,-269.0,-269.0,1.0,0.0,0.0,0.0,1.013867,0.266036,0.019003,0.00136,14.0,0.071429,ETH
poliernase574,1.0,0.994440,NaN,NaN,12.0,12.0,0.0,0.0,0.0,0.0,1.003036,0.111277,0.008560,0.05000,13.0,0.076923,ETH


In [4]:
path = '../private_data/gr14_report/raw_data_20220629-224520_raw_data.hdf5'
contribs_df = pd.read_hdf(path, "metabase")
contribs_df.head(2)

,created_on,modified_on,profile_for_clr_id,success,wallet_address,handle,raw_amount_in_token,amount_in_token,amount_to_gitcoin_in_token,amount_in_usdt,token,ip_addr
0,2022-06-09 00:00:30.127718+00:00,2022-06-09 00:10:41.355795+00:00,264900,True,0x2c07a2ac29541b9124da12836729d6305494d6f0,hossynn,1.00,1.00,0.05,1.00,DAI,217.146.94.20
1,2022-06-09 00:00:30.995488+00:00,2022-06-09 23:14:59.593242+00:00,264900,False,0x2c07a2ac29541b9124da12836729d6305494d6f0,hossynn,0.05,0.05,0.05,0.05,DAI,217.146.94.20


In [5]:
cond_4 = ~pd.isnull(output_df.aggregate_score)
cond_4a = output_df.loc[cond_4, 'aggregate_score'] > 0.5

In [6]:
gr14_contributions = pd.read_csv("../private_data/gr14_report/contributions_gr14_all.csv")
print(len(gr14_contributions))
gr14_contributions.head(2)

634721


,Unnamed: 0,created_on,modified_on,success,handle,grant_id
0,0,2022-06-23 23:59:59.976253+00:00,2022-06-24 00:13:33.028389+00:00,True,ts380940,5688
1,1,2022-06-23 23:59:58.979817+00:00,2022-06-24 23:12:37.533060+00:00,False,seongjae-lim,12


In [19]:

contribs = (gr14_contributions.assign(is_sybil=contribs_df.handle.map(cond_4a))
                 .dropna(subset=['is_sybil'])
                 .assign(kind='original')
                 .assign(match='t')
     )

print(len(contribs))
contribs_with_sybil = contribs[contribs.is_sybil]
print(len(contribs_with_sybil))

contribs_with_sybil.head(2)

623400
167988


,Unnamed: 0,created_on,modified_on,success,handle,grant_id,is_sybil,kind,match
0,0,2022-06-23 23:59:59.976253+00:00,2022-06-24 00:13:33.028389+00:00,True,ts380940,5688,True,original,t
1,1,2022-06-23 23:59:58.979817+00:00,2022-06-24 23:12:37.533060+00:00,False,seongjae-lim,12,True,original,t


In [20]:
grants = pd.read_csv("../private_data/gr14_report/grants_contributed_to_in_gr14_with_count.csv", index_col=0)
grants.set_index('id', drop=True)
grants.head(2)

,id,title,active,contribution_count,url,created_on,modified_on
0,12,Gitcoin Grants Official Matching Pool Fund,True,78714,https://gitcoin.co/grants/12/gitcoin-grants-of...,2018-12-19 22:18:55+00:00,2022-07-01 14:25:25.131326+00:00
1,13,"ethers.js - Complete, Simple and Tiny",True,3892,https://gitcoin.co/grants/13/ethersjs-complete...,2018-12-19 22:58:36+00:00,2022-07-01 14:12:07.560033+00:00


In [21]:
grant_count = contribs_with_sybil.groupby('grant_id').size().reset_index(name='sybil_contributions')
grants_sybil_contribs = grant_count.merge(grants, left_on='grant_id', right_on='id').drop('id', 1)
print(len(grants_sybil_contribs))

1008


/tmp/ipykernel_660327/2319697484.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  grants_sybil_contribs = grant_count.merge(grants, left_on='grant_id', right_on='id').drop('id', 1)


In [22]:
grants_sybil_contribs.sort_values(by=['sybil_contributions'], ascending=False, inplace=True)
grants_sybil_contribs = grants_sybil_contribs.assign(percentage = lambda df: df.sybil_contributions / df.contribution_count)
grants_sybil_contribs = grants_sybil_contribs.loc[:, ['grant_id', 'title', 'active', 'sybil_contributions', 'contribution_count', 'percentage' ,'url']]
print(grants_sybil_contribs['sybil_contributions'].sum())
non_active = grants_sybil_contribs.loc[~grants_sybil_contribs.active, :]
active = grants_sybil_contribs.loc[grants_sybil_contribs.active, :]

print(f"total: {len(grants_sybil_contribs)}")
print(f"Active: {len(active)}")
print(f"Non Active: {len(non_active)}")

active.to_csv("still_active_grants_that_recieved_sybil_contributions.csv", index=False)
non_active.to_csv("non_active_grants_that_recieved_sybil_contributions.csv", index=False)


167988
total: 1008
Active: 981
Non Active: 27


In [23]:
active

,grant_id,title,active,sybil_contributions,contribution_count,percentage,url
0,12,Gitcoin Grants Official Matching Pool Fund,True,20885,78714,0.265328,https://gitcoin.co/grants/12/gitcoin-grants-of...
699,5688,JediSwap: First AMM on StarkNet enabling insta...,True,10192,37997,0.268232,https://gitcoin.co/grants/5688/jediswap-first-...
562,5007,Lenster,True,8882,32891,0.270043,https://gitcoin.co/grants/5007/lenster
428,4268,zkRollups.xyz,True,6712,26066,0.257500,https://gitcoin.co/grants/4268/zkrollupsxyz
10,37,Connext Network,True,6055,22693,0.266822,https://gitcoin.co/grants/37/connext-network
...,...,...,...,...,...,...,...
209,1836,Blockchain Development and Consulting in Canad...,True,1,12,0.083333,https://gitcoin.co/grants/1836/blockchain-deve...
559,4981,Greenpill digital campfire,True,1,4,0.250000,https://gitcoin.co/grants/4981/greenpill-digit...
212,1870,Rebase Community - Rebase 区块链技术社区,True,1,5,0.200000,https://gitcoin.co/grants/1870/rebase-communit...
555,4967,Loot Places for Gaming on Web Pages,True,1,6,0.166667,https://gitcoin.co/grants/4967/loot-places-for...


In [24]:
non_active

,grant_id,title,active,sybil_contributions,contribution_count,percentage,url
758,5995,Quest3,False,2781,10308,0.269790,https://gitcoin.co/grants/5995/quest3
677,5511,ExSwap & Avelin Protocol,False,1328,4827,0.275119,https://gitcoin.co/grants/5511/exswap-avelin-p...
430,4276,Dune Dashboards for Aztec Protocol,False,913,3283,0.278099,https://gitcoin.co/grants/4276/dune-dashboards...
782,6075,277dao (Chinese Collaborators & Builders for D...,False,342,977,0.350051,https://gitcoin.co/grants/6075/277dao-chinese-...
877,6256,0xScope Protocol,False,99,430,0.230233,https://gitcoin.co/grants/6256/0xscope-protocol
299,3062,Atlantis World,False,67,299,0.224080,https://gitcoin.co/grants/3062/atlantis-world
494,4804,GiveStation DAO,False,47,152,0.309211,https://gitcoin.co/grants/4804/givestation-dao
709,5793,Secret Shared Validator: Distributed Validator...,False,19,71,0.267606,https://gitcoin.co/grants/5793/secret-shared-v...
190,1661,EtherDrops,False,16,17,0.941176,https://gitcoin.co/grants/1661/etherdrops
920,6406,"Adventure Gold (AGLD), the Universial Basic In...",False,16,39,0.410256,https://gitcoin.co/grants/6406/adventure-gold-...
